# General Description:

For CDD

Plotting choice as a function of delay_amount, subdivded by delay_wait_time and immediate_amount

Importing libraries and mounting Google Drive

In [5]:
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob
import os,sys


In [6]:
# global parameters
WAIT_DICT = {'now':0,'in 1 day': 0.0333333333333333,'in 1 week':0.25,'in 1 month': 1, 'in 3 months':3, 'in 1 year': 12, 'in 3 months': 3, 'in 5 years':60}

Defining count_tuples() function to return the unique items and their frequencies 

In [7]:
def resync_df(df):
    print('Presync')
    for c in list(df):
        if ('amount' in c) or ('delay' in c):
            if c == 'SS amount':
                continue
            tabulate_col(df,fn='/tmp',col=c,save=False,verbose=True)
    df = df.rename(columns={'SS amount' : 'cdd_immed_amt','LL amount':'cdd_delay_amt','SS delay':'cdd_immed_wait','LL delay':'cdd_delay_wait','Response [-1O_0I_1D]':'cdd_choice'})
    for c in ['cdd_immed_amt','cdd_delay_amt']:
        df[c] = df[c].map(lambda x: x.lstrip('gain $').replace(',','')).astype(int)
    df = df.drop(columns=['Stimulus onset','Response time','Now loc [0L_1R]'])
    for c in ['cdd_immed_wait','cdd_delay_wait']:
        df[c] = df[c].map(WAIT_DICT)
    df['cdd_trial_type'] = 'task'
    return df
  
def tabulate_col(df,fn='/tmp/',col='cdd_immed_amt',save=False,verbose=True):
    count_df = df[col].value_counts().sort_index()
    count_df = count_df.reset_index()
    if verbose:
        print(count_df)
    #create .csv file with this info
    if save:
        print("Saving to: {}".format(fn))
        count_df.to_csv(fn)

def sync_and_save(split_dir,fn,cols=[],header=0,save=False):
    try:
        df = pd.read_csv(fn,header=header)
    except Exception as e:
        print('We got an Exception: {}'.format(e))
        print('We will move on the next spreadsheet')
        return

    subj_cdd_dir = os.path.join(split_dir,get_subject(fn),'cdd')
    if not os.path.exists(subj_cdd_dir):
        print('Making subjects crdm directory : {}'.format(subj_cdd_dir))
        os.makedirs(subj_cdd_dir)
    df = resync_df(df)
    subj_cdd_fn = os.path.join(subj_cdd_dir,'{}_cdd.csv'.format(get_subject(fn)))
    print('Resynced and saving to : {}'.format(subj_cdd_fn))
    df.to_csv(subj_cdd_fn,index=0)

def get_subject(fn):
    fileparts = os.path.basename(fn).split('_')
    subj_nb = '_'.join(fileparts[1:3])
    subj = '23_DDT_{}'.format(subj_nb)
    return subj


In [8]:
raw_csv_dir = '/Volumes/UCDN/datasets/DDT/raw_csv/'
split_dir = '/Volumes/UCDN/datasets/DDT/split/'
cols = ['cdd_immed_wait','cdd_delay_amt','cdd_delay_wait','cdd_immed_amt']

#get set of all good data files for analysis
good_files = sorted(glob(os.path.join(raw_csv_dir, '*/AdjAmt*.csv')))
if (not good_files):
    print("No good files available. Check file path.")
    sys.exit()

for i, subj_fn in enumerate(good_files):
    print('Working on file: {}'.format(subj_fn))
    sync_and_save(split_dir,subj_fn,cols=cols,header=4,save=False)

Working on file: /Volumes/UCDN/datasets/DDT/raw_csv/002/AdjAmt_002_1_10-26-2017_14h-59m.csv
We got an Exception: Error tokenizing data. C error: Expected 7 fields in line 47, saw 8

We will move on the next spreadsheet
Working on file: /Volumes/UCDN/datasets/DDT/raw_csv/002/AdjAmt_002_1_10-26-2017_16h-57m.csv
Presync
     LL amount  count
0  gain $1,000     98
      SS delay  count
0   in 1 month      9
1    in 1 year      3
2  in 3 months      6
3          now     80
      LL delay  count
0   in 1 month     17
1    in 1 year     23
2  in 3 months     20
3   in 5 years     26
4          now     12
Resynced and saving to : /Volumes/UCDN/datasets/DDT/split/23_DDT_002_1/cdd/23_DDT_002_1_cdd.csv
Working on file: /Volumes/UCDN/datasets/DDT/raw_csv/002/AdjAmt_002_2_11-20-2017_13h-24m.csv
We got an Exception: Error tokenizing data. C error: Expected 7 fields in line 47, saw 8

We will move on the next spreadsheet
Working on file: /Volumes/UCDN/datasets/DDT/raw_csv/002/AdjAmt_002_2_11-20-2017_